In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [43]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key='chat_history', # template 안에 memory가 history를 저장하도록 한 곳
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"), # 누가 보냈는 지 알 수 없는, 예측하기 어려운 메세지를 가질 수 있음 (system, human, ai). Memory class로 대체될 것임
    ("human", "{question}")
])

def load_memory(input):
    # print(input)
    return memory.load_memory_variables({})['chat_history'] # 첫번째 아이템의 output, 두번째 아이템의 input

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def add_memory(input, output):
    # db에 저장할 수도 있음
    memory.save_context({"input": input}, {"output": output})

def invoke_chain(question):
    result = chain.invoke({
        "question": question # 첫번째 아이템의 input
    })
    add_memory(question, result.content)
    print(result.content)


In [44]:
invoke_chain("My name is Nico")

Hello Nico! How can I assist you today?


In [45]:
invoke_chain("What is my name?")

Your name is Nico.


In [46]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Nico'),
  AIMessage(content='Hello Nico! How can I assist you today?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Nico.')]}